In [17]:
#!pip install cufflinks

In [7]:
import yfinance
import numpy as np
import pandas as pd
import pandas
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import cufflinks as cf
cf.set_config_file(offline=True)

import warnings
warnings.filterwarnings("ignore")


In [32]:
from ActMng import AccountManager

In [33]:
starting_capital = 100000
allocation = 0.8

symbol = '1140.SR'

long_trend_days = 12
short_trend_days = 6

In [34]:
def fetch_data(symbol:str='1140.SR', start_date:str='2021', end_date:str=None):
    
    if end_date is None:
        end = pandas.Timestamp.now()
    else:
        end = end_date
    
    return pdr.get_data_yahoo(symbol, pandas.Timestamp(start_date), pandas.Timestamp(end))[['Open','Close']]
df = fetch_data(symbol)

[*********************100%***********************]  1 of 1 completed


In [35]:
fast_trend = df.Open.rolling(window=short_trend_days).mean()
slow_trend = df.Open.rolling(window=long_trend_days).mean()

df['ma_fast'] = fast_trend
df['ma_slow'] = slow_trend
df[['Close','ma_fast','ma_slow']].iplot()

In [36]:
df['trade_direction'] = np.sign(fast_trend-slow_trend)
df

,Open,Close,ma_fast,ma_slow,trade_direction
Date,,,,,
2021-01-03,28.299999,28.200001,NaN,NaN,NaN
2021-01-04,28.200001,28.100000,NaN,NaN,NaN
2021-01-05,28.100000,28.299999,NaN,NaN,NaN
2021-01-06,28.250000,28.049999,NaN,NaN,NaN
2021-01-07,28.000000,28.299999,NaN,NaN,NaN
...,...,...,...,...,...
2021-11-04,43.000000,43.400002,42.433333,42.574999,-1.0
2021-11-07,43.500000,43.500000,42.824999,42.620833,1.0
2021-11-08,43.400002,44.000000,43.058333,42.587500,1.0


In [37]:
#Determine the direction of the moving average differences
df_trades = df['trade_direction'].dropna().diff().dropna().apply(np.sign).astype('int8')
df_trades = pandas.DataFrame(df_trades[df_trades !=0])

#Label our intended trades
df_trades['trade'] = df['trade_direction'].map({1:'Long',-1:'Short'})
df['trades'] = df_trades['trade']


In [38]:
#Label our opening trade after removing our window period
df['trades'].replace(pandas.np.NaN,'None',inplace=True)
df = df.dropna()
df['trades'].iloc[0]=('Long' if df['trade_direction'].iloc[0]==1.0 else 'Short')
df

,Open,Close,ma_fast,ma_slow,trade_direction,trades
Date,,,,,,
2021-01-18,28.799999,28.850000,28.599999,28.400000,1.0,Long
2021-01-19,28.700001,28.850000,28.691666,28.433333,1.0,None
2021-01-20,28.850000,28.650000,28.741666,28.487500,1.0,None
2021-01-21,28.650000,28.350000,28.741666,28.533333,1.0,None
2021-01-24,28.299999,28.200001,28.683333,28.537500,1.0,None
...,...,...,...,...,...,...
2021-11-04,43.000000,43.400002,42.433333,42.574999,-1.0,None
2021-11-07,43.500000,43.500000,42.824999,42.620833,1.0,Long
2021-11-08,43.400002,44.000000,43.058333,42.587500,1.0,None


In [39]:
# Our trading strategy implementation - feel free to brose ActMng.py if you are curious.
trading_strategy = AccountManager(symbol=symbol,trades_df=df,starting_capital=starting_capital,allocation_amount=allocation)
results = trading_strategy.run()

In [40]:
#Here we can see a day by day breakdown of the account
results['result_df']

,Open,Close,ma_fast,ma_slow,trade_direction,trades,account_val,cash_bal,position,position_changes,fees
2021-01-18,28.799999,28.850000,28.599999,28.400000,1.0,Long,100137.892179,19998.938119,2777.78,2777.78,1.0
2021-01-19,28.700001,28.850000,28.691666,28.433333,1.0,None,100137.892179,19998.938119,2777.78,0.00,0.0
2021-01-20,28.850000,28.650000,28.741666,28.487500,1.0,None,99582.334060,19998.938119,2777.78,0.00,0.0
2021-01-21,28.650000,28.350000,28.741666,28.533333,1.0,None,98749.002179,19998.938119,2777.78,0.00,0.0
2021-01-24,28.299999,28.200001,28.683333,28.537500,1.0,None,98332.336239,19998.938119,2777.78,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2021-11-04,43.000000,43.400002,42.433333,42.574999,-1.0,None,98364.892099,181908.593036,-1924.97,0.00,0.0
2021-11-07,43.500000,43.500000,42.824999,42.620833,1.0,Long,98170.398036,19633.323036,1805.45,3730.42,2.0
2021-11-08,43.400002,44.000000,43.058333,42.587500,1.0,None,99073.123036,19633.323036,1805.45,0.00,0.0
2021-11-09,44.950001,46.000000,43.508333,42.741666,1.0,None,102684.023036,19633.323036,1805.45,0.00,0.0


In [41]:
# Here are our simulated orders as we would submit to an API endpoint for algorthmic trading. 
results['broker_orders']

,symbol,date,time,order,order_type,price,size,exchange_fees
2021-01-18,1140.SR,2021-01-18 00:00:00,At Market Open,BUY,MARKET,28.799999,2777.78,1.0
2021-01-26,1140.SR,2021-01-26 00:00:00,At Market Open,SELL,MARKET,28.299999,2777.78,1.0
2021-01-26,1140.SR,2021-01-26 00:00:00,At Market Open,SELL,MARKET,28.299999,2787.54,1.0
2021-02-11,1140.SR,2021-02-11 00:00:00,At Market Open,BUY,MARKET,28.049999,2787.54,1.0
2021-02-11,1140.SR,2021-02-11 00:00:00,At Market Open,BUY,MARKET,28.049999,2832.20,1.0
2021-03-24,1140.SR,2021-03-24 00:00:00,At Market Open,SELL,MARKET,33.099998,2832.20,1.0
2021-03-24,1140.SR,2021-03-24 00:00:00,At Market Open,SELL,MARKET,33.099998,2745.73,1.0
2021-04-04,1140.SR,2021-04-04 00:00:00,At Market Open,BUY,MARKET,35.599998,2745.73,1.0
2021-04-04,1140.SR,2021-04-04 00:00:00,At Market Open,BUY,MARKET,35.599998,2398.61,1.0
2021-04-15,1140.SR,2021-04-15 00:00:00,At Market Open,SELL,MARKET,34.650002,2398.61,1.0


In [46]:
results_df = results["result_df"]
results_df.head()

,Open,Close,ma_fast,ma_slow,trade_direction,trades,account_val,cash_bal,position,position_changes,fees
2021-01-18,28.799999,28.850000,28.599999,28.400000,1.0,Long,100137.892179,19998.938119,2777.78,2777.78,1.0
2021-01-19,28.700001,28.850000,28.691666,28.433333,1.0,None,100137.892179,19998.938119,2777.78,0.00,0.0
2021-01-20,28.850000,28.650000,28.741666,28.487500,1.0,None,99582.334060,19998.938119,2777.78,0.00,0.0
2021-01-21,28.650000,28.350000,28.741666,28.533333,1.0,None,98749.002179,19998.938119,2777.78,0.00,0.0
2021-01-24,28.299999,28.200001,28.683333,28.537500,1.0,None,98332.336239,19998.938119,2777.78,0.00,0.0


In [48]:
#Todo:
# Calculate the Absolute Return of our trend-following trading strategy
absolute_return_tf =  results_df["account_val"].iloc[-1] / starting_capital - 1
print(f"Absolute Return = {absolute_return_tf:.5f}")
# Calculate the CAGR or our trend-following trading strategy
CAGR_tf = (results_df["account_val"].iloc[-1] / starting_capital) ** (252/len(df)) - 1
print(f"CAGR = {CAGR_tf:.5f}")

# Calculate the IR (Sharpe without RF-rate) of our trend-following trading strategy
information_ratio_tf = (results_df["account_val"].pct_change().mean() / results_df["account_val"].pct_change().std()) * (252) ** 0.5
print(f"IR = {information_ratio_tf:.5f}")

#Plot the equity curve (hint: use 'account_val')
# Todo
results_df["account_val"].iplot()

Absolute Return = 0.02684
CAGR = 0.03343
IR = 0.25406


In [49]:
#Todo:
# Calculate the Absolute Return of a buy-and-hold strategy in the same period
absolute_return_bh = (results_df["Close"].iloc[-1] / results_df["Close"].iloc[0]) - 1
print(f"Absolute Return = {absolute_return_bh:.5f}")


# Calculate the CAGR or our trend-following trading strategy
CAGR = (results_df["Close"].iloc[-1] / results_df["Close"].iloc[0]) ** (252/len(df)) - 1
print(f"CAGR = {CAGR:.5f}")


# Calculate the IR (Sharpe without RF-rate) of our trend-following trading strategy
information_ratio = (results_df["Close"].pct_change().mean() / results_df["Close"].pct_change().std()) * np.sqrt(252)
print(f"IR = {information_ratio:.5f}")


#Plot the equity curve (hint: use 'close' returns and your initial capital value to generate an equity curve)
# Todo
(results_df["Close"].pct_change().add(1).cumprod()  * starting_capital * allocation + 20_000).iplot()

Absolute Return = 0.59445
CAGR = 0.78451
IR = 2.45081


In [ ]:
#Todo - optional, below:
# Copy any of the code from above to new cells below, and see how changing the symbol, dates and trend parameters changes the performance
# Can you beat the reference implementation?